## Зачистка данных.

✅️

Проблемы:

- Если товар ни разу не продавался в магазине, есть смысл его удалить. Уменьшим дисбаланс, та и просто данных станет меньше, полегче будет.
НО! Если мы удаляем этот товар, то потом, на продакшене, должны отлавливать запрос на прогноз по этому товару в этом магазине и выдавать 0.

- Под одним ID в разных магазинах может быть разный товар. Нельзя передавать эти числа в модель как есть сейчас. Преобразовать идентификаторы товаров в категориальные признаки вида "s40-i35" (магазин №40, товар №35).
Цитата из описания даты: Some of the products may be a similar item (such as milk) but have a different id in different stores/regions/suppliers.

- Идентификаторы магазинов нельзя передавать в модель как число. Преобразовать "40" в "s40", рассматривать как категориальный признак. Иначе модель будет пытаться искать логику в том, что магазин №40 > №30.


### Загружаем данные

In [8]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('./data/train.csv')

display(train_df.head())
display(train_df.info())

,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0
3,2012-01-01,1,4,0
4,2012-01-01,1,5,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4617600 entries, 0 to 4617599
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   date       object
 1   store_nbr  int64 
 2   item_nbr   int64 
 3   units      int64 
dtypes: int64(3), object(1)
memory usage: 140.9+ MB


None

### Вместо идентификатором магазина и товара использовать коды.

In [9]:
train_df['store_code'] = 's' + train_df['store_nbr'].astype(str)
train_df['store_item_code'] = (
    train_df['store_nbr'].astype(str) + '-' + train_df['item_nbr'].astype(str)
)

### Добавить признаки продаж за предидущие дни.

Сортируем внутри каждой пары магазин-товар по дате;

Создаём новые колонки:
- units_yesterday - продажи того же товара за вчера
- units_prev_week - продажи за прошлую неделю
- rolling_mean_4w – скользящее среднее продаж за 4 недели (28 дней).

Потом снова сортировать всё по дате

In [10]:
# Сортировка для корректных лагов
train_df = train_df.sort_values(['store_item_code', 'date'])

# Генерация лагов и скользящего среднего
grouped = train_df.groupby('store_item_code')['units']

train_df['units_yesterday'] = grouped.shift(1).fillna(0)
train_df['units_prev_week'] = grouped.shift(7).fillna(0)

# 4 недели = 28 дней; min_periods=1 чтобы первые записи не были NaN
train_df['rolling_mean_4w'] = (
    grouped.rolling(window=28, min_periods=1).mean().reset_index(level=0, drop=True)
)

# Порядок колонок и восстановление сортировки
cols = ['date', 'store_code', 'store_item_code', 'store_nbr', 'item_nbr', 'units',
        'units_yesterday', 'units_prev_week', 'rolling_mean_4w',]
train_df = train_df[cols].sort_values(['date', 'store_item_code'])

### Удалить вообще не продающиеся товары в магазинах

Сгруппировать по магазину и товару, посчитали суммарные продажи. Выбрать пары, где сумма = 0. Через merge‑фильтр выбросить все записи, относящиеся к этим «мертвым» парам.

In [11]:
# 1. ищем «глухие» пары магазин‑товар
dead_pairs = (
    train_df.groupby(['store_nbr', 'item_nbr'], as_index=False)['units']
            .sum()
            .query('units == 0')[['store_nbr', 'item_nbr']]
)

# 2. отбрасываем все строки с такими парами
clean_df = (
    train_df
      .merge(dead_pairs, on=['store_nbr', 'item_nbr'], how='left', indicator=True)
      .query('_merge == "left_only"')
      .drop(columns='_merge')
      .reset_index(drop=True)
)

In [12]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236038 entries, 0 to 236037
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             236038 non-null  object 
 1   store_code       236038 non-null  object 
 2   store_item_code  236038 non-null  object 
 3   store_nbr        236038 non-null  int64  
 4   item_nbr         236038 non-null  int64  
 5   units            236038 non-null  int64  
 6   units_yesterday  236038 non-null  float64
 7   units_prev_week  236038 non-null  float64
 8   rolling_mean_4w  236038 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 16.2+ MB


In [13]:
clean_df

,date,store_code,store_item_code,store_nbr,item_nbr,units,units_yesterday,units_prev_week,rolling_mean_4w
0,2012-01-01,s1,s1-i28,1,28,2,0.0,0.0,2.000000
1,2012-01-01,s1,s1-i40,1,40,0,0.0,0.0,0.000000
2,2012-01-01,s1,s1-i47,1,47,0,0.0,0.0,0.000000
3,2012-01-01,s1,s1-i51,1,51,1,0.0,0.0,1.000000
4,2012-01-01,s1,s1-i89,1,89,0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
236033,2014-10-31,s9,s9-i105,9,105,0,0.0,0.0,0.000000
236034,2014-10-31,s9,s9-i42,9,42,0,0.0,0.0,0.000000
236035,2014-10-31,s9,s9-i45,9,45,57,30.0,56.0,44.928571
236036,2014-10-31,s9,s9-i5,9,5,34,82.0,29.0,36.285714


In [14]:
clean_df.to_csv('./data/train-clean.csv')